# Data gathering and preparation
This notebook with gather all the different datasets used for this project and prep them to be utilized for exploratory data analysis.

In [66]:
import pandas as pd
import numpy as np
import geopandas as gpd
import zipfile
import pickle
from sodapy import Socrata

# import api token
from src import *

### Bicycle Counters

In [5]:
b_counters = pd.read_csv('./data/Bicycle_Counters.csv',
                        index_col='id')
b_counters.sort_values(by='id')

,name,latitude,longitude,domain,site,timezone,interval,sens,installationDate,counter
id,,,,,,,,,,
0,Manhattan Bridge 2012 Test Bike Counter,40.699810,-73.985890,New York City DOT,100005020,(UTC-05:00) US/Eastern;DST,15,0,08/31/2012,NaN
1,2nd Avenue - 26th St S,40.739710,-73.979540,New York City DOT,100009424,(UTC-05:00) US/Eastern;DST,15,0,05/22/2015,NaN
2,Prospect Park West,40.671288,-73.971382,New York City DOT,100009425,(UTC-05:00) US/Eastern;DST,15,0,11/07/2016,Y2H13094304
3,Manhattan Bridge Ped Path,40.714573,-73.994950,New York City DOT,100009426,(UTC-05:00) US/Eastern;DST,15,0,12/04/2013,Y2H13074107
4,Williamsburg Bridge Bike Path,40.710530,-73.961450,New York City DOT,100009427,(UTC-05:00) US/Eastern;DST,15,0,12/03/2013,Y2H13074108
5,Ed Koch Queensboro Bridge Shared Path,40.751038,-73.940820,New York City DOT,100009428,(UTC-05:00) US/Eastern;DST,15,0,12/04/2013,Y2H19111445
6,Manhattan Bridge 2013 to 2018 Bike Counter,40.699768,-73.985820,New York City DOT,100009429,(UTC-05:00) US/Eastern;DST,15,0,12/03/2013,NaN
7,Staten Island Ferry,40.643387,-74.072075,New York City DOT,100010017,(UTC-05:00) US/Eastern;DST,15,0,03/31/2016,Y2H13094300
8,Pulaski Bridge,40.742563,-73.951492,New York City DOT,100010018,(UTC-05:00) US/Eastern;DST,15,0,06/24/2017,Y2H13094301


Bicycle Counter csv has `23` counters with name, lat/lon, site?, installation date and counter columns. Need to figure out what `counter` column is as it has some NaN

In [51]:
b_counters.to_pickle('./pickle/b_counters')

### Bicycle Counts

In [7]:
b_counts = pd.read_csv('./data/Bicycle_Counts.csv',
                      index_col='id')
b_counts

/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,counts,date,status,site
id,,,,
0,41.0,08/31/2012 12:00:00 AM,4.0,100005020
1,52.0,08/31/2012 12:15:00 AM,4.0,100005020
2,38.0,08/31/2012 12:30:00 AM,4.0,100005020
3,36.0,08/31/2012 12:45:00 AM,4.0,100005020
4,40.0,08/31/2012 01:00:00 AM,4.0,100005020
...,...,...,...,...
303835,27.0,04/30/2021 10:45:00 PM,0.0,100062893
303836,24.0,04/30/2021 11:00:00 PM,0.0,100062893
303837,23.0,04/30/2021 11:15:00 PM,0.0,100062893


Looks like Bicicyle counts has over 3 million rows dating back from 2012 through April 2021. Will need to make this an API if I want to consistenly update with latest data. 
<br>
`site` column can facilitate merge with `b_counters` dataframe to get lat/lon for mapping purposes.
<br>
I am not sure what the `status` column means. 

In [52]:
b_counts.to_pickle('./pickle/b_counts')

### Motor Vehicle Collisions

In [43]:
collisions = pd.read_csv('./data/Motor_Vehicle_Collisions_-_Crashes.csv',
                        parse_dates=['CRASH DATE'],
                        infer_datetime_format=True)
collisions.head()

/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-04-14,5:32,NaN,NaN,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4407480,Sedan,Sedan,NaN,NaN,NaN
1,2021-04-13,21:35,BROOKLYN,11217.0,40.68358,-73.97617,"(40.68358, -73.97617)",NaN,NaN,620 ATLANTIC AVENUE,...,NaN,NaN,NaN,NaN,4407147,Sedan,NaN,NaN,NaN,NaN
2,2021-04-15,16:15,NaN,NaN,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,NaN,...,NaN,NaN,NaN,NaN,4407665,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
3,2021-04-13,16:00,BROOKLYN,11222.0,NaN,NaN,NaN,VANDERVORT AVENUE,ANTHONY STREET,NaN,...,Unspecified,NaN,NaN,NaN,4407811,Sedan,NaN,NaN,NaN,NaN
4,2021-04-12,8:25,NaN,NaN,0.00000,0.00000,"(0.0, 0.0)",EDSON AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4406885,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


In [18]:
# see all rows to see all value counts in selected column
pd.set_option('display.max_rows',None)
collisions['VEHICLE TYPE CODE 1'].value_counts()

Sedan                                     439233
PASSENGER VEHICLE                         416206
Station Wagon/Sport Utility Vehicle       351132
SPORT UTILITY / STATION WAGON             180291
Taxi                                       43928
4 dr sedan                                 40035
TAXI                                       31911
Pick-up Truck                              28310
VAN                                        25266
OTHER                                      22966
UNKNOWN                                    19927
Box Truck                                  19488
Bus                                        16252
LARGE COM VEH(6 OR MORE TIRES)             14397
BUS                                        13993
SMALL COM VEH(4 TIRES)                     13216
PICK-UP TRUCK                              11505
LIVERY VEHICLE                             10481
Bike                                       10070
Tractor Truck Diesel                        8293
Van                 

In [45]:
# create list of vehicle types that count as bicycles
bike_list = ['Bike','BICYCLE','Minibike','Minicycle']

# filter dataframe for any columns in vehicle type that are in the bike list
b_collisions = collisions[(collisions['VEHICLE TYPE CODE 1'].isin(bike_list)) |\
                         (collisions['VEHICLE TYPE CODE 2'].isin(bike_list)) |\
                         (collisions['VEHICLE TYPE CODE 3'].isin(bike_list)) |\
                         (collisions['VEHICLE TYPE CODE 4'].isin(bike_list)) |\
                         (collisions['VEHICLE TYPE CODE 5'].isin(bike_list))]

In [46]:
# set row display to 20
pd.set_option('display.max_rows',20)

# preview bike list filtered dataframe
b_collisions
        

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
54,2021-04-16,11:00,QUEENS,11368.0,40.749580,-73.865410,"(40.74958, -73.86541)",NaN,NaN,100-10 ROOSEVELT AVENUE,...,Unspecified,NaN,NaN,NaN,4407792,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
93,2021-04-14,0:00,NaN,NaN,40.601864,-74.002320,"(40.601864, -74.00232)",BATH AVENUE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4407649,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
131,2021-04-14,20:10,BROOKLYN,11238.0,40.688210,-73.965830,"(40.68821, -73.96583)",WASHINGTON AVENUE,LAFAYETTE AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4407381,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
143,2021-04-13,17:55,BRONX,10452.0,40.844105,-73.923065,"(40.844105, -73.923065)",GRANT HIGHWAY,UNIVERSITY AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4407789,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN
149,2021-04-14,19:45,BROOKLYN,11201.0,40.694840,-73.983910,"(40.69484, -73.98391)",FLATBUSH AVENUE EXTENSION,JOHNSON STREET,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4407414,Sedan,Bike,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780263,2012-07-09,17:37,BROOKLYN,11222,40.720601,-73.954754,"(40.7206006, -73.9547539)",BEDFORD AVENUE,NORTH 12 STREET,NaN,...,Unspecified,NaN,NaN,NaN,198192,TAXI,BICYCLE,NaN,NaN,NaN
1780284,2012-07-02,9:46,MANHATTAN,10002,40.717724,-73.985765,"(40.7177239, -73.9857652)",DELANCEY STREET,CLINTON STREET,NaN,...,Unspecified,NaN,NaN,NaN,12187,PICK-UP TRUCK,BICYCLE,NaN,NaN,NaN
1780352,2012-07-07,18:40,NaN,NaN,40.867335,-73.822707,"(40.8673349, -73.8227066)",NaN,NaN,NaN,...,Unspecified,NaN,NaN,NaN,2912116,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN
1780353,2012-07-06,13:33,BROOKLYN,11209,40.625780,-74.024154,"(40.6257805, -74.0241544)",5 AVENUE,80 STREET,NaN,...,Unspecified,NaN,NaN,NaN,140835,PASSENGER VEHICLE,BICYCLE,NaN,NaN,NaN


In [47]:
# make all column names lower dase and replace underscores for spaces
b_collisions.columns = [col.replace(' ','_').lower() for col in b_collisions.columns]

In [48]:
# look at column dtypes and info
b_collisions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50267 entries, 54 to 1780385
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_date                     50267 non-null  datetime64[ns]
 1   crash_time                     50267 non-null  object        
 2   borough                        39819 non-null  object        
 3   zip_code                       39819 non-null  object        
 4   latitude                       46413 non-null  float64       
 5   longitude                      46413 non-null  float64       
 6   location                       46413 non-null  object        
 7   on_street_name                 43141 non-null  object        
 8   cross_street_name              37054 non-null  object        
 9   off_street_name                7024 non-null   object        
 10  number_of_persons_injured      50264 non-null  float64       
 11  number_of_pe

In [49]:
# pickle dataframe
b_collisions.to_pickle('./pickle/b_collisions')

There are a ton of null values in the motor vehicle dataframe...some are ok, others are not. For instance if we don't even have the `latitude` or `longitude` of the crash site we have no idea where the crash occurred. Consider dropping rows where any null in coordinates.

## Shapefiles

### Bike Priority Areas (shapefile)

In [67]:
# set client request for Socrata API
client = Socrata("data.cityofnewyork.us",app_token)

# make request using API endpoint
results = client.get("byz4-8k8n")

# create GeoDataFrame out of request
b_priority_gdf = gpd.GeoDataFrame(results)

In [68]:
b_priority_gdf

,boro_cd_cod,shape_leng,the_geom,shape_area
0,BK 3,36213.8362463,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",79460977.3423
1,BK 5,58026.4575893,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",156204154.973
2,BK 12,52468.9113739,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",99833185.8188
3,BK 14,49259.5617348,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",82168722.5327
4,BK 15,96010.9373532,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",131678761.813
5,BK 17,43287.2618759,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",93791381.4552
6,QN 3,37011.0141973,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",82975598.8703
7,QN 4,36820.7520393,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",65634904.8855
8,QN 5,69923.9500542,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",210416759.573
9,BK 4,37060.9166461,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",56660385.9663


### Street Improvement Projects (Corridors)

### Street Improvement Projects (Intersections)

### Cityracks

### Bicycle Routes